In [ ]:
from IPython.core.display import HTML, display
HTML("""
<style>
svg {
    width:40% !important;
    height:40% !important;
}

.container { 
    width:100% !important;
}
</style>
""")

## Erstellen des Spiels und der Spieler

Vor dem Start eines Schachspiels müssen zunächst Spiel generell sowie die partizpierenden Spieler der gewünschten Spielertypen erstellt werden. Dazu muss der Spieler dem Programm entweder Parameter mitgeben, die das Spiel direkt mit den gewünschten Spielern startet, oder aber der Spieler wird zum Programmstart nach gewünschten Einstellungen gefragt.

In beiden Fällen wird anschließend eine Liste an Spielern an Hand der gegebenen Funktionen erstellt, mittels welcher Das Spiel dann stattfinden kann. Dies findet in der Funktion `interrogate_settings` statt.

Wird das Spiel mit Parametern gestartet werden diese mit in die `interrogate_settings` Funktion übermittelt. Andernfalls werden alle fehlenden Informationen in der Funktion über die Nutzerschnittstelle abgefragt. Dies sieht wie folgt aus:

In [ ]:
def interrogate_settings(self, player_names=None, player_types=None, player_difficulty = None):
    players = []
    for i in range(2):
        num = i+1
        if player_names is None:
            name = self.get_player_name(num)
        else:
            name = player_names[i]

        if player_types is None:
            player_type = self.get_player_type(num)
        else:
            player_type = player_types[i]

        difficulty = None
        if player_type == 2 and player_difficulty is None:
            difficulty = self.get_difficulty(num)
        elif player_type == 2 and player_difficulty is not None:
            difficulty = player_difficulty[i]

        new_player = PlayerSettings(num, name, player_type, difficulty)
        players.append(new_player)

    return players

Dabei werden für jeden Spieler einmal ein Block an Abfragen durchgegangen, der alle Einstellungen abfragt und diese entsprechend speichet.

Nach der Abfrage der Informationen "Name" und "Spielertyp" wird im Falle, dass dieser Spielertyp "2", der der KI entspricht, gleicht, auch noch der Schwierigkeitsgrad abgefragt, sollte dieser nicht bereits als Startparameter übergeben wordne sein.

Anschließend wird ein neues Einstellungsparameterschema für Spieler, genannt `PlayerSettings`, erstellt und der Liste hinzugefügt. Diese Liste wird nach Durchgang der Schleife für beide Spieler zurück gegeben.

Nachdem die Einstellungen für alle Spieler bekannt sind, werden die Spieler erstellt. Dies funktioniert wie in nachfolgendem Code-Snippet zu sehen.

In [ ]:
player_settings = settings_ui.interrogate_settings()
players = []
for player_setting in player_settings:
    type = type_switcher(player_setting.type)
    players.append(type.Player(player_setting.num, player_setting.name, ui_status, player_setting.difficulty))

Dabei werden zunächst wie gehabt die Einstellungen abgefragt, ehe eine Liste aller Spieler erstellt wird. Dann wird über jeden Eintrag in der Einstellungsliste iteriert und für diesen zunächst der Spielertyp berechnet. Dabei wird der `type_switcher` zur Hilfe genommen, welcher je nach angegebenen Typ die entsprechende Klasse des Spielertyps zurückgibt. Mittels dieser wird dann ein neuer Spieler erstellt, wobei beim Initialisierren alle weiteren Daten mitgegeben werden. Dieser neue Spieler wird dann der Spielerliste hinzugefügt. Mit dieser Liste an Spielern wird dann der `ChessMaster` initialisiert, der im nächsten Kapitel beschrieben wird.

## Verwalten des Schachspiels und Pflege des Spielverlaufs

Ein essentieller Part im Erstellen eines Schachprogrammes ist erstmal das Verwalten des Schachspiels. Dabei muss garantiert werden, dass
- Solange das Spiel nicht vorbei ist, die Spieler abwechselnd einen Zug auswählen können
- Der Zug auf dem vorhandenen Schachbrett durchgeführt wird
- Das daraus entstehende Schachbrett dem Spieler sichtbar gemacht wird

Dies ist Aufgabe des `ChessMaster` und ist mit folgendem Code umgesetzt

In [ ]:
board = chess.Board()
while not board.is_game_over():
    current_player = players[int(not board.turn)]
    current_player.print_board(current_player.name, board)

    move = current_player.get_move(board)
    board.push(move)
    current_player.submit_move(move)

Dabei wird zunächst ein neues Schachbrett - hier board genannt - erstellt. Solange das Spiel auf diesem nicht vorbei ist, was mittels der `board.is_game_over()` Funktiong geprüft werden kann, wird dann stets die gleiche Schleife durchlaufen.

In dieser wird zuallererst der aktuelle Spieler ermittelt und referenziert. Dazu wird aus einer vorhandenen Liste aller Spieler derjenige gewählt, dessen Position in der Liste der gespeicherten Zugnummer des boards entspricht. Diese ist entweder 0, wenn der weiße Spieler an der Reihe ist, oder 1, wenn der schwarze Spieler den nächsten Zug auswählen kann. 

Der daraus berechnete Spieler wurde zuvor beim Spielstart ein Spielertyp zugewiesen, der auf einer Schnittstelle basiert und somit alle nötigen Funktionen implementiert. Diese wurden im Kapitel INSERT beschrieben. Im Verwalter des Schachspiels werden diese nun nach für nach aufgerufen.

Zunächst wird das board für den Spieler ausgegeben mittels der `print_board` Funktion. Ist der Spieler ein Nutzer, so wird diese für gewöhnlich am Nutzerinterface ausgegeben. Andernfalls ist dies nicht nötig und die Funktion kann leer bleiben, ohne etwas auszugeben.

Nun kommt es zum wichtigsten Teil - dem Berechnen des nächsten Zuges. Dazu wird der Spieler aufgefordert an Hand eines gegebenen boards den nächstne Zug zu nennen. In diesem Teil übernimmt beispielsweise die KI ihre Berechnungen für den nächsten Schachzug. Der Nutzer dagegen gibt diesen mittels eines Eingabefeldes ein. 

Nachdem der Zug vom entsprechenden Spieler berechnet und zurückgegeben wurde, wird dieser dem aktuellen board hinzugefügt. Dadurch wechselt auch automatisch der Spieler, der an der Reihe ist, wodurch dieser im nächsten Schleifendurchlauf nach dessen Zug gefragt wird.

Abschließend wird das neue board nochmal abgeschickt. Dabei kann zum Beispiel eine erneute Ausgabe des Schachbretts mit dem aktualisierten Zustand stattfinden oder beispielsweise bei der Nutzung einer Online-API der gewählte Zug an die Schnittstelle gesendet werden.

Nach dem Ende des Spiels wird noch das Ergebnis ausgegeben. Der Code dazu ist im folgenden Code Snippet zu sehen.

In [ ]:
result = Tools.get_board_result(board)
if result is 1:
    players[0].print_win_msg()
    players[1].print_loose_msg()
elif result is -1:
    players[1].print_win_msg()
    players[0].print_loose_msg()
else:
    players[0].print_draw_msg()
    players[1].print_draw_msg()

Dabei wird zuerst das Ergebnis an Hand der `get_board_result` Funktion aus der Hilfsklasse "Tools" ermittelt. Diese sieht wie folgt aus:

In [ ]:
if board.is_variant_loss():
    return -1 if board.turn == chess.WHITE else 1
elif board.is_variant_win():
    return 1 if board.turn == chess.WHITE else -1
elif board.is_variant_draw():
    return 0
# Checkmate.
if board.is_checkmate():
    return -1 if board.turn == chess.WHITE else 1
# Draw claimed.
if board.can_claim_draw():
    return 0
# Seventyfive-move rule or fivefold repetition.
if board.is_seventyfive_moves() or board.is_fivefold_repetition():
    return 0
# Insufficient material.
if board.is_insufficient_material():
    return 0
# Stalemate.
if not any(board.generate_legal_moves()):
    return 0
return 0

Dabei wird jede mögliche Option, wie das Spiel zum Ende gekommen sein kann, durchgegangen und anschließend das jeweilige Ergebnis zurückgegeben. Eine 1 steht für einen Sieg für weiß, eine -1 für einen Sieg für schwarz und eine 0 für ein Unentschieden beziehungsweise einen Patt.

Nach Abfrage des Ergebnisses werden beide Spieler abhängig von diesem entweder dazu aufgefordert eine Siesesbenachrichtigung auszugeben oder aber eine Benachrichtigung über die Niederlage oder ein Unentschieden.

Zusätzlich zu dem Verwalten des Spiels ist es auch Aufgabe des `ChessMaster`s die Historie aller Spiele zu pflegen. Dazu wird zunächst eine Liste von Schachbrettern der Klasse `chess.Board` angelegt. Nach jedem durchgeführten Zug wird der neue Zustand des boards zu dieser hinzugefügt. Dabei wird das Schachbrett in der fen-Notation gespeichert, die in einem String den exakten Zustand des Schachbretts wiedergeben kann. Dabei wird jedoch nur der erste Teil dieser Notation gespeichert, da dieser alleine bereits Aufschluss über die Positionierungen gibt. Die darauf folgenden Teile sind zum Speichern des Spielers, der am Zug ist, wie viele Züge bereits durchgeführt wurden und weitere Informationen, die zum Bewerten in der Historie nicht notwendig sind. 

Das Speichern der Züge im Verlauf des Spiels sieht dann wie folgt aus

In [ ]:
board = chess.Board()
turn_list = list()

while not board.is_game_over():
    current_player = players[int(not board.turn)]
    current_player.print_board(current_player.name, board)

    move = current_player.get_move(board)
    board.push(move)
    current_player.submit_move(move)
    
    turn_list.append(board.fen().split(" ")[0])

In Zeile 2 wird die Liste erstellt und in der letzten Zeile der Zustand in diese Liste gespeichert. Nach Ende des Spiels muss diese Liste noch in die bestehende Historie eingepflegt werden. Dies ist in folgendem Code-Snippet zu sehen.

In [ ]:
def groom_board_history(self, final_board, turn_list):
    victory_status = Tools.get_board_result(final_board)

    new_turn_dict = dict.fromkeys(turn_list, victory_status)

    history = pd.read_csv(HISTORY_FILE_LOC)
    history_dict = dict(zip(list(history.board), list(history.value)))

    merged_history_dict = { k: new_turn_dict.get(k, 0) + history_dict.get(k, 0) for k in set(new_turn_dict) | set(history_dict) }
    merged_history = pd.DataFrame(list(merged_history_dict.items()), columns=['board','value'])

    merged_history.to_csv(HISTORY_FILE_LOC)

Dabei wird zuerst das Ergebnis ermittelt mittels der bereits vorgestellten Funktion `get_board_result`. Danach wird ein Dictionary angelegt, das auf der einen Seite jeden gespeicherten Zustand des einzupflegenden Spiels als Schlüssel enthält und auf der anderen Seite als Wert den Ausgang des Spiels. Zur Erinnerung - dieser beträgt 1 bei Sieg von weiß, 0 bei Unentschieden/Patt und -1 bei Sieg von Schwarz.

Nachdem das Dictionary der neu einzupflegenden Züge angelegt ist, wird die vorhandene Historie aus der entsprechenden Datei ausgelesen. Dies wird mittels der in `panda` enthaltenden Funktion `pd.read_csv` durchgeführt. Anschließend wird auch daraus ein Dictionary erstellt, indem die Zeile "board" als Schlüssel und die Zeile "value" als Wert verwendet wird.

Nun werden die beiden Dictionaries zusammengefügt. Dazu wird für jeden Schlüssel aus dem Dictionary der neu einzupflegenden Spielzüge oder dem vorhandenen Historie-Dictionary der berechnete Wert aus ersterem auf den vorhandenen Wert in der Spielhistorie aufaddiert. Dadurch erhalten wird ein Dictionary, das alle Schachzustände aus der Historie sowie den neuen Spielzuständen vereint und dessen Werte durch Addition kombiniert, wodurch der neu berechnete Wert Aufschluss über den wahrscheinlichen Sieger ausgehend von einem bestimmten Zustand geben kann.

Anschließend wird dieses Dictionary in ein Panda Dataframe umgewandelt, wobei der Schlüssel für die Zeile "board" und der Wert für die Zeile "value" verwendet wird, um das DataDrame anschließend wieder zu der CSV-Datei zu speichern.

Dies ermöglicht nicht nur eine Verwaltung des Spiels, sondern gleichzeitig auch eine Speicherung aller möglichen Zustände, die bei den Evaluierungsfunktionen (beschrieben in INSERT) zur Hilfe genommen werden können

## Implementierung des "Iterative Deepening"

Die zentrale Aufgabe der KI ist es, den best möglichen Zug für eine gegebene Situation zu finden. Dabei geht diese so vor, dass sie alle möglichen Züge durchgeht und die daraus entstehenden Zustände analysiert und evaluiert. Dabei jedoch zählt nicht nur der direkt erreichbare Zustand zählt, sondern auch die aus diesem Zustand erreichbaren Zustände und so weiter. Aus diesem Grund wird immer bis zu einer bestimmten Tiefe in die Züge hineingeschaut und die sich daraus ergebenden Zustände evaluiert.

Um dies jedoch nicht fest immer bis zu einer bestimmten Tiefe durchgehen zu lassen, sondern variabel anzupassen, je nachdem wie viele Züge von dem gegebenen Zustand aus möglich sind, kann ein Zeitlimit dienen. Dabei wird anfangs die Tiefe auf 1 gesetzt und dann mittels des Minimax-Algorithmus die Züge evaluiert. Danach wird die Tiefe um 1 erhöht und erneut der Minimax-Algorithmus angewandt. Dies wird solange wiederholt, bis die angegebene Zeit abgelaufen ist. Dieser Algorithmus nennt sich "Iterative Deepening". Nähere Informationen dazu sind in Kapitel INSERT aufgeführt. 

Dem Algorithmus muss dazu der aktuelle Zustand sowie eine maximale Tiefe mitgegeben werden. Ist diese erreicht bricht der Algorithmus ab, unabhängig davon, ob das Zeitlimit überschritten ist oder nicht. Zunächst müssen beim Ausführen dann einige Werte wie folgt festgelegt werden.

In [ ]:
depth = 1
    
start_time = int(time.time())
end_time = start_time + self.time_limit
current_time = start_time

player = bool(board.turn)
self.best_possible_result = self.get_best_possible_result(board, player)

Zunächst wird die Starttiefe auf 1 festgesetzt. Danach wird die Startzeit auf die aktuelle Zeit gesetzt und die Endzeit berechnet, indem auf die Startzeit das Zeitlimit addiert wird. Zudem wird der erste Wert für die aktuelle Zeit auf die Startzeit festgelegt.

Anschließend wird für den Spieler, der aktuell am Zug ist, berechnet, was das bestmöglich zu erreichende Resultat ist. Dies wird mit der Funktion `get_best_possible_result` durchgeführt. Dies ist dazu gut, um finale Zustände dahingehend zu evaluieren, ob diese für den Nutzer die best mögliche Option ist (Sieg oder Unentschieden wenn Sieg nicht mehr möglich ist) und dementsprechend zu bewerten. Die Funktion sieht wie folgt aus.

In [ ]:
def get_best_possible_result(self, board, player):
    if player and board.has_insufficient_material(chess.WHITE):
        return 0
    if not player and board.has_insufficient_material(chess.BLACK):
        return 0
    if player and not board.has_insufficient_material(chess.WHITE):
        return 1
    if not player and not board.has_insufficient_material(chess.BLACK):
        return -1

Dabei muss der Funktion der aktuelle Zustand sowie der Spieler, der an der Reihe ist, mitgegeben werden. Ist der aktuelle Spieler der der weißen Figuren (player == True) und hat weiß unzureichende Materialien für einen Sieg, so ist der bestmögliche Zustand ein Patt. Das gleiche Ergebnis wird zurückgegeben, wenn der Spieler der der schwarzen Figuren ist (player == False) und schwarz unzureichende Materialien hat.

Ist der Spieler jedoch weiß und er hat noch ausreichend Materialien, so wird der Wert 1 zurückgegeben, da ein Sieg noch erreichbar ist. Genauso wird für den schwarzen Spieler der Wert -1 zurückgegeben, falls er noch ausreichende Materialien besitzt, da dieser noch einen Sieg erreichen kann und der Wert -1 für einen Sieg von Schwarz steht.

Nach dieser Abfrage wird der eigentliche Algorithmus des "Iterative Deepening" durchgeführt.

In [ ]:
legal_moves = list(board.legal_moves)
while current_time < end_time and depth <= max_depth:
    move_val_dict = {}

    best_value = float('-inf')
    best_move = legal_moves[0]

    for move in legal_moves:
        tmp_board = chess.Board(str(board.fen()))
        tmp_board.push(move)

        value = self.min_value(str(tmp_board.fen()), player, float('-inf'), float('inf'), depth - 1, end_time)
        move_val_dict[move] = value

        if value == MAX_BOARD_VALUE:
            return move
        if value > best_value:
            best_value = value
            best_move = move

    legal_moves.sort(key=move_val_dict.get, reverse=True)
    depth += 1
    current_time = int(time.time())

Dabei wird zunächst eine Liste aller legalen Züge erstellt. Dann wird eine Schleife so lange durchlaufen, bsi entweder die Zeit abgelaufen ist oder aber die maximale Tiefe erreicht ist. 

In dieser Schleife wird ein Dictionary aller Züge mit ihren berechneten Werte erstellt. Zudem werden Anfangswerte für die besten Züge und dessen Wert festgelegt. Der Anfangswert des besten Zugs wird auf den ersten Zug festgesetzt. Der Wert dieses wird auf den Wert "- Unendlich" gesetzt.

Nun wird über alle legalen Züge iteriert und für jeden Zug ein temporärer "board" angelegt, das erreichbaren Zustand widerspiegelt. Nun wird mittels des Minimax-Algorithmus der Wert dieses boards ermittelt. Dabei wird als Alpha "- Unendlich" und als Beta "Unendlich" mitgegeben. Was die Werte Alpha und Beta aussagen, wird im Kapitel INSERT beschrieben. Zudem wird die Tiefe auf einen Wert festgelegt, der um einen Wert geringer ist als die maximale Tiefe, da durch Aufruf der Funktion in die erste Tiefe hineingegangen wurde. Zudem wird die Zeit mitgegeben, zu der der Algorithmus enden muss, damit der Minimax Algorithmus dementsprechend endet und das zeitlimit nicht überschreitet.

Nachdem der Minimax-Algorithmus fertig durchlaufen ist, wird der Wert mit dem Zug zu dem Dictionary hinzugefügt. Gleicht der Wert dem maximalen Wert für einen Zustand, ist also dementsprechend ein Sieg, wird der Zug direkt zurückgegeben, da mit diesem dann auf jeden Fall ein Sieg erreich werdne kann. Andernfalls wird der Wert verglichen, ob er besser ist als der aktuelle Wert. Ist dies der Fall, so wird der neue beste Wert auf den aktuell berechneten festgelegt, ebenso wie der beste Zug auf den der aktuellen Iteration gesetzt wird.

Nachdem alle Züge durchlaufen wurden, wird die Liste aller legalen Züge an Hand der berechneten Werte sortiert, damit im nächsten Durchlauf die Züge in dieser Reihenfolge durchlaufen werden. Dies verbessert den Durchsatz beim Alpha Beta Pruning, wie in Kapitel INSERT beschrieben und garantiert zudem, dass der beste Wert der vergangenen Runde gewählt wird, falls der Minimax-Algorithmus beim Durchlaufen der nächst tieferen Tiefe das Zeitlimit erreicht, bevor die Runde komplett evaluiert werden konnte.

Abschließend wird noch die Tiefe um 1 erhöht und die aktuelle Zeit auf die Systemzeit gesetzt, damit an Hand dieser entschieden werden kann, ob der Algorithmus noch weiter durchlaufen darf.

Nachdem dann die Zeit abgelaufen ist und alle Züge in der für die angegebenen Zeit maximalen Tiefe evaluiert wurden, wird der best mögliche Zug zurückgegeben. Dieser wird dann von der Ki ausgeführt.

## Implementierung des Minimax-Algorithmus mit Alpha-Beta-Pruning

Um den bestmöglichen Zug zu erkennen, wird beim Minimax Algorithmus jeder Zug bis zu einer gewissen Tiefe betrachtet. Dabei wird unter der Prämisse gehandelt, dass auch der Gegner stets den besten Zug macht. Dies führt dazu, dass der bestmöglichste Zug ausgewählt wird (max), der erreichbar ist, wenn der Gegner mit dem für ihn jeweils besten Zug antwortet, der für den Spieler somit der schlechteste ist (min). Genauer ist dies in Kapitel INSERT erklärt.

Die Umsetzung dabei erfolgt in zwei Funktionen - `min_value` und `max_value`. Erstere berechnet dabei den schlecht möglichsten Ausgang für den Spieler aus einer bestimmten Position, also den besten Ausgang für den Gegner. Letztere berechnet den best möglichsten Ausgang. Beide Funktionen sind sehr ähnlich aufgebaut und in folgendem Code-Snippet zu sehen.

In [ ]:
def min_value(self, board_fen, player, alpha, beta, depth, time_limit):
    board = chess.Board(board_fen)
    v = float('inf')

    if board.is_game_over() or depth == 0:
        return self.evaluate_board(board, player)
    if int(time.time()) >= time_limit:
        return float("-inf")

    for move in board.legal_moves:
        tmp_board = chess.Board(board_fen)
        tmp_board.push(move)
        v = min(v, self.max_value(str(tmp_board.fen()), player, alpha, beta, depth -1, time_limit))
        if v <= alpha:
            return v
        beta = min(beta, v)
    return v

Neben dem akteullen Zustand des Spiels in fen Notation und dem Spieler, für den es den Zustand zu evaluieren gilt, wird außerdem eine Tiefe sowie ein Zeitlimit mitgegeben sowie die Werte Alpha und Beta. Alpha und Beta sind dabei, wie in Kapitel INSERT erklärt, dazu da, um den Minimax-Algorithmus zu beschleunigen, indem nicht jeder mögliche Zustand betrachtet wird. Durch diese Werte fallen nämlich solche weg, die direkt als irrelevant betrachtet werden können, da ohnehin bereits ein besserrer (im Fall von `min_value`) bzw. schlechterer (im Fall von `max_value`) Wert gefunden wurde.

Nachdem das Schachbrett über die fen-Notation erstellt wurde, wird überprüft, ob das Spiel bereits vorbei ist oder die mitgegebene Tiefe auf 0 liegt. In beiden Fällen wird der aktuelle Zustand direkt evaluiert und zurückgegeben. Ansonsten wird geprüft, ob das übergebene Zeitlimit bereits erreicht wurde. In dem Fall wird der maximal negative Wert zurück gegeben, damit dieser Zug keine weitere Beachtung mehr bei der endgültigen Auswahl findet.

Ist auch dies nicht der Fall, werden alle von dem gegebenen Zustand aus erreichbaren Zustände durchgegangen. Dazu wird zunächst ein temporäres Schachbrett - hier board - erstellt und der Zug auf diesem ausgeführt. Dies ist dann der neue, erreichbare Zustand. Dieser wird dann in die nächst tiefere Iteration gegeben, bei der nun der maximale Wert gesucht wird. Dabei wird auch der Spieler mitgegeben sowie die Werte Alpha und Beta und das Zeitlimit ebenso wie die um eins reduzierte Tiefe. Ist dabei ein Wert dabei, der kleiner ist als das aktuelle v, wird dieser Wert als das neue v genommen. Andernfalls bleibt v beim akteullen Wert.

Anschließend wird überprüft, ob der Wert v kleiner ist als der aktuelle Alpha Wert. Ist dies der Fall, muss der Pfad keine weitere Beachtung finden und es kann direkt v zurück gegeben werden. ist dies nicht der Fall, so wird Beta auf v gesetzt, falls dieser Wert kleiner als das aktuelle Beta ist. Nach Durchgang aller legalen Züge wird dann der sich aus all diesen Iterationen ergebende Wert v zurück gegeben.

Die `max_value` Funktion läuft similar ab, mit dem einzigen Unterschied, dass hier der maximale statt der minmale Wert gesucht wird und dementsprechend die Vergleiche sowie Startwerte angepasst sind. Außerdem gibt dieser bei der um eins tieferen Iteration in die `min_value` Funktion, an der Stelle, an der diese in die `max_value` Funktion gibt. Der restliche Aufbau bleibt unverändert.

In [ ]:
def max_value(self, board_fen, player, alpha, beta, depth, time_limit):
    board = chess.Board(board_fen)
    v = float('-inf')

    if board.is_game_over() or depth == 0:
        return self.evaluate_board(board, player)
    if int(time.time()) >= time_limit:
        return float("inf")

    for move in board.legal_moves:
        tmp_board = chess.Board(board_fen)
        tmp_board.push(move)
        v = max(v, self.min_value(str(tmp_board.fen()), player, alpha, beta, depth -1, time_limit))
        if v >= beta:
            return v
        alpha = max(alpha, v)
    return v

Mit diesem Algorithmus wird ein Baum aus allen Pfaden erstellt, an Hand der der beste Zug ermittelt werden kann wie in Kapitel INSERT beschrieben. Da es sich jedoch nur selten um Endzustände handelt, für die eine Bewertung trivial erfolgen kann, ist eine Evaluierung der Zustände nötig. Diese wird im nächsten Kapitel beschrieben.

## Evaluierung eines gegebenen Schachbretts

Um gegebene Zustände auch mitten im Spiel bewerten zu können, müssen diese an Hand bestimmter Kriterien bewertet werden können, die über Sieg oder Niederlage hinausgehen. Dazu gibt es verschiedene Ansätze, wie in Kapitel INSERT erläutert. Einige davon wurden im Laufe des Projektes umgesetzt und implementiert. Diese werden in diesem Kapitel erläutert. Zunächst jedoch gilt es aufzuzeigen, wie die Evaluierung der Zustände im generellen umgesetzt wird.

Zunächst wird zum Evaluieren eines Zustandes jener Zustand sowie der Spieler, für den dieser zu evaluieren ist, in die Funktion `evaluate_board` gegeben, die die Evaluierung zentral verwaltet. 

In [ ]:
def evaluate_board(self, board, player):
    player_color = chess.WHITE if player else chess.BLACK

    if board.is_game_over():
        result = Tools.get_board_result(board)
        if result is self.best_possible_result:
            return MAX_BOARD_VALUE
        if result is -1:
            return -1 * MAX_BOARD_VALUE

    evaluation_val = 0
    for func, value in self.evaluation_funcs_dict.items():
        if value > 0:
            evaluation_val = evaluation_val + value * func(board, player_color)
    return evaluation_val

Dabei wird zunächst an Hand des Spielers die Farbe dieses ermittelt, die später bei den einzelnen Evaluierungsfunktionen benötigt wird.

Dann wird für den Fall, dass der gegebene Zustand einem Endzustand gleicht, das Ergebnis dieses ermittelt. Gleicht dieses dem bestmöglichen Ergebnis, dass mittels der `get_board_result` Funktion zuvor ermittelt wurde (siehe INSERT), so wird der maximale Wert (Unendlich) für den Zusand zurückgegeben. Ist das Ergebnis des übermittelten Zustands jedoch eine Niederlage, so wird der maximale Wert umgekehrt und zurückgegeben (minus Unendlich). 

Gleicht der Zustand keinem Endzustand, so werden bestimmte Evaluationsfunktionen durchlaufen und aufaddiert. Dazu startet der Wert bei 0 und für jede Evaluierungsfunktion wird das Ergebnis dieser multipliziert mit einem festgelegten Faktor zu dem Gesamtwert aufaddiert. Die Faktoren sowie die durchzuführenden Evaluierungsfunktionen sind dabei abhängig vom Schwierigkeitsgrad der KI sowie vom Spielstatus (Eröffnung, Mittelspiel, Endspiel).

Dabei werden zur Performanz-Steigerung jedoch nur Evaluierungsfunktionen durchgegangen, dessen Faktor höher als 0 liegt. Der Grund dafür ist, dass bestimmte Funktionne je nach Spielstatus leichter aus der Evaluierung herausgenommen werden können, ohne, dass das gesamte Dictionary angepasst werden muss und zudem keine unnötige Rechenzeit durch Berechnung eines Werts benötigt wird, der im Endeffekt ohnehin nicht zum Evaluierungswert aufaddiert wird.

Der daraus entstehende Evaluierungswert, der zurückgegeben wird, gibt einen guten Aufschluss über den Wert des aktuellen Zustands. Dazu werden verschiedene Evaluierungsfunktionen verwendet, wie in folgendem Abschnitt zu sehen ist.

### Materialbewertung

Eine zentrale sowie einfache Bewertung ist dabei die Bewertung der vorhandenen Materialien auf dem Spielfeld. Dabei werden alle Figuren der jeweiligen Spieler zusammengezählt und je nach Figur mit einem Wert multipliziert. 

Dabei ist zunächst jedem Figurentyp ein Wert zuzuweisen. Üblicherweise werden Bauern dabei 1 Punkt, Türmen 5 Punkte, Springern sowie Läufern jeweils 3 Punkte und der Dame 9 Punkte zugeordnet. Diese werden dann zusammengerechnet.

In [ ]:
def get_value_by_color(color):
    attacked_pieces_value = map(lambda piece_type : len(board.pieces(piece_type, color)) * assign_piece_value(piece_type), chess.PIECE_TYPES)
    return sum(attacked_pieces_value)

def get_board_value():
    white_value = get_value_by_color(chess.WHITE)
    black_value = get_value_by_color(chess.BLACK)
    
    return white_value - black_value if color is chess.WHITE else black_value - white_value

Um den Gesamtwert des Schachbretts zu berechnen muss zunächst der Wert aller weißer Figuren berechnet werden und von diesem der Wert aller schwarzen Figuren abgezogen werden. Je nach angegebenen Spieler wird für diesen ein positiver Wert zurückgegeben, wenn das Spiel zu dessen Gunsten verläuft und ein negativer, wenn dies nicht der Fall ist.

Damit die Werte der Spieler berechnet werden können, wird die Anzahl aller Figurentypen der jeweiligen Farbe berechnet und diese mit dem Wert der Figurentypen multipliziert. Am Ende werden die Ergebnisse für alle Figurentypen zusammengezählt und zurückgegeben.

### Materialbewertung attackierter Figuren

Die Berechnung der attackierten Figuren ist ähnlich zu dem Vorgehen bei der Berechnung des Brettwerts. Dabei werden erst die Werte, der vom weißen Spieler attackierten Figuren berechnet und davon die Werte der vom schwarzen Spieler attackierten Figuren abgezogen. Auch hierbei ist ein positives Ergebnis zum Vorteil des angegebenen Spielers und ein negativer Wert zum Vorteil des Gegenübers. Ebenso gilt umso höher der Wert, desto deutlicher der Vorteil.

In [ ]:
def get_attacked_pieces_value_by_color(attacker_color, defender_color):
    attackedSquares = filter(lambda square : board.is_attacked_by(attacker_color, square) and not board.piece_at(square) is None and board.piece_at(square).color is defender_color, chess.SQUARES)
    attackedPieces = map(lambda square : board.piece_at(square).piece_type, attackedSquares)
    value = map(assign_piece_value, attackedPieces)
    return sum(value)
     
def get_attacked_pieces_value():
    white_value = get_attacked_pieces_value_by_color(chess.WHITE, chess.BLACK)
    black_value = get_attacked_pieces_value_by_color(chess.BLACK, chess.WHITE)
    
    return white_value - black_value

Um diese Werte der attackierten Figuren zu berechnen wird jedes Feld durchgegangen. Daraus werden die Felder gefiltert, die von einer Figur der Farbe des Verteidigers belegt sind und von einer Figur der angreifenden Farbe attackiert werden können. Anschließend wird zu diesen Feldern der Typ der Figur zugeordnet, die sich auf dem Feld befindet. Daraufhin werden diesen ihre jeweiligen Werte zugeordnet und diese abschließend summiert.

### Positionsbewertung

Um die Positionen der einzelnen Figuren zu bewerten, werden zunächst für jeden Figurentypen Matrizen benötigt, die über jedes Feld eine Aussage über den Wert der Position der Figur geben. Diese sind in Kapitel INSERT einsehbar. An Hand dieser Matrizen wird dann Aussage über den Wert getroffen.

Dabei wird zunächst jedes Feld auf dem Schachbrett durchgegangen und die darauf befindliche Figur berechnet. Dies wird mittels einer verschachtelten Schleife gelöst, die zunächst alle Reihen durchgeht und dann die einzelnen Felder in dieser Reihe.

In [ ]:
def get_board_positions_value_by_color(board, color):
    sum = 0
    for rank in range(0,8):
        for file in range(0,8):
            piece = board.piece_at(chess.square(file, rank))
            if (piece and piece.color == color):
                piece_pos_value = get_position_value_by_square(board, rank, file, color)
                sum += piece_pos_value
    return sum

Nachdem die Figur ermittelt wurde wird diese, falls diese der angegebenen Farbe angehört, gemeinsam mit den Werten für Reihe und Spalte an die Funktion `get_position_value_by_square` übergeben. Nachdem diese den Wert zurückgegben hat wird dies zu der bisherigen Summe aufaddiert und am Ende die Summe aller Figuren zurück gegeben.

In der Funktion `get_position_value_by_square` wird mittels der Matrix der Wert der Figur an der gegebenen Position ermittelt.

In [ ]:
def get_position_value_by_square(board, rank, file, color):
    piece_type = board.piece_type_at(chess.square(file, rank))
    piece_matrix = assign_piece_matrix(piece_type) if color == chess.BLACK else np.flip(assign_piece_matrix(piece_type))
    piece_pos_value = piece_matrix[rank,file]
    return piece_pos_value

Dazu wird zunächst der Typ der Figur auf dem gegebenen Feld ermittelt. Dann wird die dazu passende Matrix bestimmt und entsprechend gespiegelt, falls die Farbe des angegebenen Spielers weiß sein sollte, damit die Matrix mit den Positionen aus der Sicht des Spielers übereinstimmt.

Schlussendlich wird der Wert in der Matrix über die Reihe und Spalte ermittelt und zurück gegeben. 

### Königszonen-Sicherheit

Ein weiterer, wichtiger Wert ist die Sicherheit des Königs bemessen an Hand der Figuren, die dessen Zone angreifen. 

Dazu wird diese Zone berechnet und dann alle Figuren, die diese Zone angreifen ermittelt. Mittels der in Kapitel INSERT vorgestellten Berechnung wird dann der Wert des Angriffs auf die Königszone berechnet. 

In [ ]:
def calculate_king_zone_safety(board, color):
    attacker_color = chess.WHITE if color == chess.BLACK else chess.BLACK
    king_zone = calculate_king_zone(board, color)
    attackers = get_attackers_by_squares(board, king_zone, attacker_color)
    attack_weight = get_king_attack_weight(len(attackers))
    value_of_attack = 0
    for attacker in attackers:
        value_of_attack += get_king_attack_constants(attacker.piece_type)
    
    return (value_of_attack * attack_weight) / 1000

Dabei wird zuerst die Farbe des Angreifers berechnet, indem die gegebene Farbe umgekehrt wird. Danach wird mittels der Funktion `calculate_king_zone` die Königszone berechnet. Diese Funktion gibt eine Menge von Feldern zurück, die der Königszone angehören.

In [ ]:
def calculate_king_zone(board, color):
    king_zone = chess.SquareSet()
    king_rank, king_file = get_piece_position(board, chess.Piece(chess.KING, color))

    rank_range = range(0,4) if color == chess.WHITE else range(-3, 1)
    for rank_summand in rank_range:
        if (king_rank + rank_summand) in range(0, 8):
            for file_summand in range(-1, 2):
                if (king_file + file_summand) in range (0,8):
                    king_zone.add(chess.square(king_file + file_summand, king_rank + rank_summand))
    
    return king_zone

Um dies zu ermöglichen, wird zunächst die Position des Königs ermittelt. Dabei werden alle Felder durchgegangen bis der König der entsprechenden Farbe gefunden wurde. Dann wird Reihe sowie Spalte zurückgegeben.

Nun werden alle Felder der Königszone einzeln durchgegangen und zu der Menge an Feldern hinzugeführt. Dabei werden bei den Spaltel alle Felder bis 3 Felder in Richtung des Gegners durchgegangen und für jede Spalte ein Feld links bis zu einem Feld rechts von dem König mitgezählt. Dabei wird zuvor jeweils überprüft, ob sich die Spalte beziehungsweise die Reihe noch au fdem Spielfeld befinden. Ist dies der Fall, wird das Feld der Menge hinzugefügt und diese wird am Ende zurück gegeben.

Als nächstes wir ddann für diese Menge an Feldern ermittelt, welche Figuren diese Felder angreifen. Dies wird mittels der `get_attackers_by_squares` Funktion durchgeführt.

In [ ]:
def get_attackers_by_squares(board, square_set, attacker_color):
    attacker_dict = {}
    for square in square_set:
        attacker_square_set = board.attackers(attacker_color, square)
        for attacker_square in attacker_square_set:
            attacker_piece = board.piece_at(attacker_square)
            if not (attacker_piece.piece_type is chess.PAWN or attacker_piece.piece_type is chess.KING):
                attacker_dict[attacker_piece] = attacker_dict.get(attacker_piece, 0) + 1
    return attacker_dict

Dabei wird jedes Feld einzeln durchgegangen und für alle jeweils eine Menge an Figuren erstellt, die dieses Feld angreifen. Alle Angriefer werden dann einzeln durchgegangen und dessen Figurentyp ermittelt. Wenn dies weder ein Bauer noch ein König ist, wird die Figur einem Dictionary von allen Angreifern hinzugefügt. Der Wert dieses Eintrags setzt sich aus der Anzahl zusammen, wie viele Felder von dieser Figur angegriffen werden. Dieses Dictionary wird nach Durchgang jedes Feldes zurückgegeben.

Nachdem dieses Dictionary zurückgegeben wurde, wird die Gewichtung der Attacke ermittelt. Dabei wird die Anzahl der verschiedenen Figuren, die die Zone angreifen, zur Hilfe genommen und ein ensprechender Wert zurückgegeben, wie in Kapitel INSERT beschrieben.

Abschließend wird für jeden Angreifer noch der Wert dieses ermittelt, wie ebenfalls in Kapitel INSERT beschrieben, und diese alle aufaddiert. Dieser Wird wird dann mit dem berechneten Gewicht multipliziert und durch 1000 dividiert. Das Ergebnis aus dieser Berechnung wird dann zurückgegeben und gibt einen Aufschluss über die Sicherheit des Königs. Dies kann auch für die Sicherheit des gegnerischen Königs angewandt werden, indem schlicht die Farbe des Angreifers auf die eigene Farbe gesetzt wird.

### Mobilität

### Bewertung an Hand angelegter Historie

Besonders am Anfang des Spiels ist es oft unschlüssig, wie man ein Schachbrett bewerten kann, da noch sehr viele Optionen des Spielverlaufs offen sind. Dabei kann eine angelegte Historie helfen, die Aufschluss über Siegchancen geben kann. Wie eine solche Historie angelegt werden kann, wurde in Kapitel INSERT besprochen.

In [ ]:
def get_board_value_by_history(board, color):
    dataset = pd.read_csv(HISTORY_FILE_LOC)
    row = dataset.loc[dataset['board'] == board.fen().split(" ")[0]]
    value = row['value'].item() if len(row['value']) == 1 else 0
    return value if color is chess.WHITE else -1*value

Abgefragt werden kann diese, indem zunächst die Daten aus der Datei geladen werden. Dann wird die Reihe abstrahiert, dessen Wert in der Spalte "board" dem mitgegebenen Zustand (in fen Notation konvertiert) gleicht. Dazu wird der entsprechende Wert ausgelesen und zurückgegeben. Beim Zurückgeben wird der Wert noch negiert, falls der angegebene Spieler der der schwarzen Figuren ist, da in der Historie die Werte aus Sicht des Spielers der weißen Figuren gespeichert wird. 